In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 2. リポジトリのディレクトリへ移動
# ※パスはご自身の環境に合わせてください
PROJECT_PATH = "/content/drive/MyDrive/git/ips_compensation"
%cd {PROJECT_PATH}

/content/drive/MyDrive/git/ips_compensation


In [3]:
%pwd

In [4]:
%rm -rf out_opt
!python ips_compensation4_stacks_envelope_AstepIso_v2.py \
  --outdir out_opt \
  --stack LC_AC_abs \
  --Amin 0.0 --Amax 3.0 --Ad 0.05 \
  --ReCmin -280 --ReCmax 280 --ReCd 5 \
  --theta 30 --phi 45 \
  --track_progress --progress_plot \
  --progress_iso_stride 5 \
  --export_envelope \
  --export_Astep_iso \
  --export_ReA_ReC_map



=== Running LC/A/C (abs) ===

Saved all outputs to: /content/drive/MyDrive/git/ips_compensation/out_opt
Progress logs saved: progress_case*.csv/.json


In [5]:
!python analyze_stokes.py \
  --fn out_opt \
  --csv progress_LC_AC_abs.csv \
  --row -1 \
  --theta 30 --phi 45 \
  --basis pol_in \
  --stage el#2_C



Off-axis analyzer axis vs. output (near-linear) polarization axis (from progress CSV)
--------------------------------------------------------------------------------
CSV      : out_opt/progress_LC_AC_abs.csv
row      : -1   (update_idx=124)
angle    : theta=30.00 deg, phi=45.00 deg
basis    : pol_in  (transverse basis u,v)
stage    : el#2_C  (reading columns: s1_el#2_C, s2_el#2_C, s3_el#2_C)
pol_in/out: 0.000 / 0.000 deg
--------------------------------------------------------------------------------
Stokes at stage (normalized):
  s1,s2,s3 = 0.965519131312, 0.246028218265, 0.000510662480
Interpretation:
  s3 ~ 0  => close to linear polarization (small ellipticity).
  (s1,s2) => determines linear-azimuth angle psi in the SAME (u,v) basis.

Axes (angles in the SAME u-v transverse plane):
  analyzer transmission azimuth alpha = 98.213210701738 deg
  output polarization azimuth   psi   = 7.147784403170 deg  (psi=0.5*atan2(s2,s1))
  delta = (alpha - psi) mod 180       = 91.065426298568 d

In [23]:
## Dispersion
!python run_dispersion.py \
  --progress_csv out_opt/progress_LC_AC_abs.csv --row -1 \
  --outdir out_disp \
  --tag LC_AC_abs \
  --lc_basis abs \
  --pol_in 0.0 --pol_out 0.0 --relA 0.25 --relLC 0.25 \
  --wl_min_nm 450 --wl_max_nm 650 --wl_step_nm 20 \
  --dispersion flat \
  --iso_every 5


In [7]:
## LC retardation
%rm -rf out_ReLC
!python run_Re_LC.py \
  --progress_csv out_opt/progress_LC_AC_abs.csv --row -1 \
  --outdir out_ReLC \
  --lc_basis abs \
  --pol_in 0.0 --pol_out 0.0 --relA 0.1 --relLC 0.25 \
  --scale_min 0.75 --scale_max 1.25 --scale_step 0.01 \
  --iso_every 0


In [8]:
## Analyzer misalignment
#!python run_rot_AC.py \
#  --progress_csv out_opt/progress_LC_AC_abs.csv --row -1 \
#  --outdir out_mis_polout \
#  --ref_A_scale 0.75 --ref_ReC_nm 90 \
#  --pol_in 0.0 --pol_out 0.0 --relA 0.25 --relLC 0.25 \
#  --rot_mode misalign \
#  --scan_target polout --delta_min -3 --delta_max 3 --delta_step 0.5 \
#  --iso_every 1


In [9]:
## LC misalignment
#!python run_rot_AC.py \
#  --progress_csv out_opt/progress_LC_AC_abs.csv --row -1 \
#  --outdir out_mis_LC \
#  --ref_A_scale 0.75 --ref_ReC_nm 90 \
#  --pol_in 0.0 --pol_out 0.0 --relA 0.25 --relLC 0.25 \
#  --rot_mode misalign \
#  --scan_target LC --delta_min -3 --delta_max 3 --delta_step 0.5 \
#  --iso_every 1


In [10]:
## A misalignment
#!python run_rot_AC.py \
#  --progress_csv out_opt/progress_LC_AC_abs.csv --row -1 \
#  --outdir out_mis_A \
#  --pol_in 0.0 --pol_out 0.0 --relA 0.25 --relLC 0.25 \
#  --rot_mode misalign \
#  --scan_target A --delta_min -3 --delta_max 3 --delta_step 0.5 \
#  --iso_every 1


In [19]:
!rm -rf out_mis*

In [20]:
## A+polout
!python run_rot_AC_updated.py \
  --progress_csv out_opt/progress_LC_AC_abs.csv --row -1 \
  --outdir out_mis_Apolout \
  --pol_in 0.0 --pol_out 0.0 --relA 0.25 --relLC 0.25 \
  --rot_mode A_polout \
  --delta_min -3 --delta_max 3 --delta_step 0.5 \
  --iso_every 1


In [24]:
!rm *.tar
!tar --warning=no-file-changed -cvf out_opt.tar out_opt
!tar --warning=no-file-changed -cvf out_mis_Apolout.tar out_mis_Apolout
!tar --warning=no-file-changed -cvf out_disp.tar out_disp



out_opt/
out_opt/progress_LC_AC_abs.json
out_opt/progress_LC_AC_abs.csv
out_opt/progress_LC_AC_abs_CR.png
out_opt/iso_progress_LC_AC_abs/
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u000.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u005.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u010.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u015.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u020.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u025.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u030.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u035.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u040.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u045.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u050.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u055.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u060.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u065.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u070.png
out_opt/iso_progress_LC_AC_abs/iso_LC_AC_abs_u075.pn